# **Return of the Schema** for *ARCO Italian Cultural Heritage*


## Path Definition Basic Elements

In [ ]:
from rdflib import Graph, RDF, RDFS, OWL, Namespace
from urllib.parse import quote
from rdflib.namespace import split_uri
from rdflib.term import URIRef
from pathlib import Path
import pickle
import csv
import ast
import json

def serialize(graph, path):
    graph.serialize(path.with_suffix(".xml"), format="xml")
    !/home/navis/robot/robot merge --input {path.with_suffix(".xml")} --output {path.with_suffix(".owl")}
    path.with_suffix(".xml").unlink()

In [ ]:
MATERIALIZE = True
VERSION = 5
DATASET_NAME = f"ARCO-{str(VERSION)}"
DATASET_NAME += f"-{'MATERIALIZE' if MATERIALIZE else "BASE"}"
BASE_FILENAME = f"filtered{VERSION}_triples.txt"
CA_FILENAME = f"class_assertions_filtered{VERSION}.json"

home_path = Path().cwd().absolute().parent.parent 
dataset_path = home_path / "kgsaf_data" / f"{'materialize' if MATERIALIZE else "base"}" / "unpack" / DATASET_NAME
arco_path = home_path / "kgsaf_data" / "ontologies"/ "unpack" / "ARCO-2025"
arco_triples_file = arco_path / "ARCO_ABOX" / BASE_FILENAME
arco_ca_file = arco_path / "ARCO_ABOX" / CA_FILENAME

print("Base Path", home_path)
print("Ontology", arco_path)
print("Dataset", dataset_path)
print("CA", arco_ca_file)
print("TRIPLE", arco_triples_file)

print("")

if MATERIALIZE:
    print("MATERIALIZED Ontology")
    onto_file = arco_path / "arco_merged_repaired_materialized.owl"
else:
    print("BASE Ontology")
    onto_file = arco_path / "arco_merged_repaired.owl"

print("Loading Ontology")

arco_ontology = Graph()
arco_ontology.parse(onto_file, format="xml")

print("Ontology Loaded")

# Triple Cleaning and Splitting, Deprecated Check

In [ ]:
with open(arco_triples_file, "r") as in_f, open(arco_path / "ARCO_ABOX" / f"filtered{version}_removed.txt", "w") as out_f, open(arco_path / "removed_uris.txt", "r" ) as rem_f:
    r_list = {s.strip("\n") for s in rem_f.readlines()}
   
    for line in in_f.readlines():
        sline = line.strip().split("\t")
        s = sline[0]
        p = sline[1]
        o = sline[2]
        if (p in r_list) or (o in r_list) or (s in r_list) or (URIRef(s), RDF.type, OWL.Class) in arco_ontology or (URIRef(o), RDF.type, OWL.Class) in arco_ontology:
            print(f"Removed {s,p,o}")
        else:
            out_f.write(f"{s}\t{p}\t{o}\n")


In [ ]:
import pykeen
from pykeen.triples import TriplesFactory
from pykeen.triples.splitting import CoverageSplitter
import numpy as np
from rdflib import Graph, OWL, Literal
from rdflib.namespace import XSD    

triples = TriplesFactory.from_path(arco_path / "ARCO_ABOX" / f"filtered{version}_removed.txt")

In [ ]:
real_obj_props = {str(s) for s in set(arco_ontology.subjects(RDF.type, OWL.ObjectProperty)) - BUILTIN_URI }

print(len(real_obj_props))

In [ ]:
obj_props = set(triples.relation_id_to_label.values())
print(len(obj_props))
print(obj_props)

reals = real_obj_props & obj_props

print(len(reals))

In [ ]:
import pykeen
from pykeen.triples import TriplesFactory
from pykeen.triples.splitting import CoverageSplitter
import numpy as np

MIN_TRIPLES_RELATION = int(VERSION)


rels, counts = np.unique(triples.mapped_triples[:, 1], return_counts=True)
rel_counts = dict(zip(rels, counts))

keep_relations = [r for r, c in rel_counts.items() if c >= MIN_TRIPLES_RELATION]

triples_clean = triples.new_with_restriction(
    relations=keep_relations
)

In [ ]:
print("Original triples:", triples.num_triples)
print("Cleaned triples:", triples_clean.num_triples)

In [ ]:
entity_mappings = {v:k for k,v in triples_clean.entity_id_to_label.items()}
relation_mappings = {v:k for k,v in triples_clean.relation_id_to_label.items()}

In [ ]:
train, valid, test = triples_clean.split(
    ratios=[0.7, 0.1, 0.2],
    random_state=42,
    method=CoverageSplitter(),      
)

In [ ]:
train_clean = TriplesFactory.from_labeled_triples(
    triples=train.triples,
    entity_to_id=entity_mappings,
    relation_to_id=relation_mappings
)

valid_clean = TriplesFactory.from_labeled_triples(
    triples=valid.triples,
    entity_to_id=entity_mappings,
    relation_to_id=relation_mappings
)

test_clean = TriplesFactory.from_labeled_triples(
    triples=test.triples,
    entity_to_id=entity_mappings,
    relation_to_id=relation_mappings
)

In [ ]:
print(train_clean)
print(test_clean)
print(valid_clean)

In [ ]:
from pykeen.triples.leakage import unleak

train_unleak, valid_unleak, test_unleak = unleak(
    train_clean,
    *[valid_clean, test_clean],
    n=None,
    minimum_frequency=0.97
    )

In [ ]:
print(train_unleak)
print(test_unleak)
print(valid_unleak)


In [ ]:
(dataset_path / "abox" / "splits").mkdir(exist_ok=True, parents=True)


targets = [
    (dataset_path / "abox/splits/train", train_unleak.triples),
    (dataset_path / "abox/splits/valid", valid_unleak.triples),
    (dataset_path / "abox/splits/test", test_unleak.triples)
]


for path, split in targets:
    out_graph = Graph()
    for triple in split:
        s = URIRef(triple[0].strip())
        p = URIRef(triple[1].strip())
        o = URIRef(triple[2].strip())
        out_graph.add((URIRef(s), URIRef(p), URIRef(o)))

    out_graph.serialize(path.with_suffix(".nt"), format="nt")

!cat {dataset_path}/abox/splits/*.nt > {dataset_path}/abox/triples.nt

# ABOX Individuals and Class Assertions

In [ ]:
data_triples = Graph()
data_triples.parse(dataset_path / "abox" / "triples.nt")

individuals = set(data_triples.subjects()) | set(data_triples.objects())

print("Len Individuals", len(individuals))
del data_triples

In [ ]:
out_graph = Graph()

for ind in individuals:
    out_graph.add((ind, RDF.type, OWL.NamedIndividual))

serialize(out_graph, dataset_path / "abox" / "individuals")
del out_graph

### [BASE] RDF Lib Class Assertions

In [ ]:
out_graph = Graph()

with open(arco_ca_file, "r") as ca_file:
    ca_data = json.load(ca_file)
    for key in ca_data.keys():
        ind = URIRef(key.strip())
        for c in ca_data[key]:
            cls = URIRef(c.strip())
            if (cls, RDF.type, OWL.Class) in arco_ontology:
                out_graph.add((ind, RDF.type, cls))

serialize(out_graph, dataset_path / "abox" / "class_assertions")

### [REASONED] Intermediate Reasoning File

In [ ]:
out_graph = Graph()

with open(arco_ca_file, "r") as ca_file:
    ca_data = json.load(ca_file)
    for key in ca_data.keys():
        ind = URIRef(key.strip())
        for c in ca_data[key]:
            cls = URIRef(c.strip())
            if (cls, RDF.type, OWL.Class) in arco_ontology:
                out_graph.add((ind, RDF.type, cls))

serialize(out_graph, dataset_path / "abox" / "unreasoned_class_assertions")

In [ ]:
print(onto_file)
!java -Xmx16G -jar /home/navis/robot/robot.jar merge -vvv \
    --input {dataset_path / "abox" / "unreasoned_class_assertions.owl"} \
    --input {dataset_path / "abox" / "individuals.owl"} \
    --input {dataset_path / "abox" / "triples.nt"} \
    --input {onto_file} \
    --output {dataset_path / "abox" / "arco_10_intermediate_abox_tbox.owl"}

### [REASONED] Server Reasoned Class Assertions

In [ ]:
print("Loading Reasoned Class Assertions from ", arco_path / "ARCO_CA" / f"arco_intermediate_abox_tbox_ca_materialized.owl" )
ca_graph = Graph()
ca_graph.parse(arco_path / "ARCO_CA" / f"arco_intermediate_abox_tbox_ca_materialized.owl")

In [ ]:
out_graph = Graph()

for ind in individuals:
    for o in set(ca_graph.objects(ind, RDF.type)) - BUILTIN_URI:
        out_graph.add((ind,RDF.type, o))

serialize(out_graph, dataset_path / "abox" / "class_assertions")

# RBOX Roles Definition and Properties

In [ ]:
data_triples = Graph()
data_triples.parse(dataset_path / "abox" / "triples.nt")

class_assertions = Graph()
class_assertions.parse(dataset_path / "abox" / "class_assertions.owl")

In [ ]:
seed_obj_props = set(data_triples.predicates())
print("Seed Object Properties", len(seed_obj_props))

seed_classes =  set(class_assertions.subjects(RDF.type, OWL.Class))
print("Seed Classes", len(seed_classes))

In [ ]:
from rdflib import Graph, URIRef, BNode, RDF, RDFS, OWL

BUILTIN_URI = {
    URIRef("http://schema.org/Thing"),
    OWL.Thing,
    OWL.Nothing,
    OWL.NamedIndividual,
    OWL.Class,
    OWL.topObjectProperty,
    OWL.bottomObjectProperty,
    RDF.type,
    RDFS.domain,
    RDFS.range,
    OWL.ObjectProperty,
    OWL.Restriction,
    OWL.DatatypeProperty,
    RDFS.Literal
}

removal = [
    URIRef("http://www.w3.org/ns/prov#wasDerivedFrom"),
    RDFS.isDefinedBy,
    URIRef("http://www.w3.org/ns/prov#wasInfluencedBy")
]



def extract_recursive_description(graph: Graph, elements: URIRef) -> Graph:

    extracted_graph = Graph()
    elem_to_process = set(elements)
    processed = set()

    while elem_to_process:

        e = elem_to_process.pop()
        processed.add(e)

        print(f"Processing {e}")

        for s,p,o in graph.triples((e, None, None)):
            extracted_graph.add((s,p,o))

            if (o not in BUILTIN_URI) and (o not in processed):

                if isinstance(o, BNode):
                    elem_to_process.add(o)

                if (o, RDF.type, OWL.Class) in graph:
                    elem_to_process.add(o)

                if (o, RDF.type, OWL.ObjectProperty) in graph:
                    elem_to_process.add(o)

                if (o, RDF.type, OWL.DatatypeProperty) in graph:
                    elem_to_process.add(o)

        
    return extracted_graph

out_graph = extract_recursive_description(arco_ontology, seed_classes | seed_obj_props)

serialize(out_graph, dataset_path / "ontology")

In [ ]:
from rdflib import BNode

onto_graph = Graph()
onto_graph.parse(dataset_path / "ontology.owl")



def extract_description(graph: Graph, elem: URIRef) -> Graph:

    extracted_graph = Graph()
    elem_to_process = {elem}
    processed = set()


    while elem_to_process:

        e = elem_to_process.pop()
        processed.add(e)

        print(f"Processing {e}")

        for s,p,o in graph.triples((e, None, None)):
            extracted_graph.add((s,p,o))

            if (o not in BUILTIN_URI) and (o not in processed):
                if isinstance(o, BNode):
                    elem_to_process.add(o)

                if (o, RDF.type, OWL.Class) in graph:
                    extracted_graph.add((o, RDF.type, OWL.Class))

                if (o, RDF.type, OWL.ObjectProperty) in graph:
                    extracted_graph.add((o, RDF.type, OWL.ObjectProperty))

                if (o, RDF.type, OWL.DatatypeProperty) in graph:
                    extracted_graph.add((o, RDF.type, OWL.DatatypeProperty))

    return extracted_graph


rbox_graph = Graph()
for prop in set(onto_graph.subjects(RDF.type, OWL.ObjectProperty)) - BUILTIN_URI:
    rbox_graph += extract_description(onto_graph, prop)

for prop in set(onto_graph.subjects(RDF.type, OWL.DatatypeProperty)) - BUILTIN_URI:
    rbox_graph += extract_description(onto_graph, prop)

(dataset_path / "rbox").mkdir(exist_ok=True)

serialize(rbox_graph, dataset_path / "rbox" / "roles")

In [ ]:
taxonomy_graph = Graph()

for c in set(onto_graph.subjects(RDF.type, OWL.Class)) - BUILTIN_URI:
    for s,p,o in onto_graph.triples((c, None, None)):
        if p == RDFS.subClassOf:
            taxonomy_graph.add((s,p,o))
            if isinstance(o, BNode):
                taxonomy_graph += extract_description(onto_graph, o)

(dataset_path / "tbox").mkdir(exist_ok=True)
serialize(taxonomy_graph, dataset_path / "tbox" / "taxonomy")

In [ ]:
schema_graph = Graph()


for c in set(onto_graph.subjects(RDF.type, OWL.Class)) - BUILTIN_URI:
    if not isinstance(c, BNode):
        for s,p,o in onto_graph.triples((c, None, None)):
            if p != RDFS.subClassOf:
                
                schema_graph.add((s,p,o))

                for elem in onto_graph.objects(o, RDF.type):
                    schema_graph.add((o, RDF.type, elem))

                if isinstance(o, BNode):
                    print(f"Found BNODE in Triple {s, p, o}")
                    schema_graph += extract_description(onto_graph, o)
            

serialize(schema_graph, dataset_path / "tbox" / "schema")

# Final Ontology and Knowledge Graph

In [ ]:
!/home/navis/robot/robot merge \
--input  {dataset_path / "ontology.owl"} \
--input  {dataset_path / "abox" / "individuals.owl"} \
--input {dataset_path / "abox" / "triples.nt"} \
--input {dataset_path / "abox" / "class_assertions.owl"} \
--output {dataset_path / "knowledge_graph.owl"}